In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
key=os.getenv('OPENAI_API_KEY')

In [5]:
                                                                                                                                                            llm= ChatOpenAI(openai_api_key=key,model_name='gpt-3.5-turbo',temperature=0.5)

c:\Users\khars\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [6]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    '1':{
        'mcq':"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
    '2':{
        'mcq':"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        'correct':"correct answer",
    },
    '3':{
        'mcq':"multiple choice question",
        "options":{
            "a":"choice here",
            "b":"choice here",
            "c":"choice here",
            "d":"choice here",
        },
        "correct":"correct answer",
    },
}

In [26]:
TEMPLATE= '''
text:{text}
You are an expert MCQ maker. Given the above text, it  is your job to \
create a quiz of {number} multiplechoice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your resonse like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}
        '''

In [27]:
quiz_generation_prompt = PromptTemplate(
    input_variables=['text','number','subject','tone','response_json'],
    template=TEMPLATE
)

In [10]:
quiz_chain = LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key='quiz',verbose= True)

c:\Users\khars\mcqgen\env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [28]:
TEMPLATE2='''
You arean expert nglish graammarian and writer. Given a multiple choice quiz for {subject} students.
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Onlu use at max 50 words for complexity.
If the quiz is not at per withthe cognitive and analytical abilities of the students,\
update thequiz questions which needs to be changed  and change the tone such that it perfectly fits the student ability.
Quiz_MCQs:
{quiz}

check from an expert English writer of the above quiz:
'''

In [29]:
quiz_evaluate_prompt = PromptTemplate(input_variables=['subject','quiz'],template = TEMPLATE) 

In [30]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluate_prompt,output_key='review',verbose=True)

In [31]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],input_variables=['text','number','subject','tone','response_json'],
                                        output_variables=['quiz','review'],verbose=True)

In [32]:
file_path=r'C:\Users\khars\mcqgen\data.txt'

In [33]:
file_path

'C:\\Users\\khars\\mcqgen\\data.txt'

In [34]:
with open(file_path,'r') as file:
    text= file.read()

In [35]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [36]:
NUMBER= 5
SUBJECT= 'machine learning'
tone='simple'

In [53]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            'text':text,
            'number': NUMBER,
            'subject':SUBJECT,
            'tone':tone,
            'response_json':json.dumps(RESPONSE_JSON)
        }
    )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[9][10] The synonym self-teaching computers was also used in this time period.[11][12]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[13] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[14] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used 

In [41]:
print(f'Total Tokens:{cb.total_tokens}')
print(f'Prompt Tokens:{cb.prompt_tokens}')
print(f'Complete Tokens: {cb.completion_tokens}')
print(f'Total cost: {cb.total_cost}')

Total Tokens:2652
Prompt Tokens:1793
Complete Tokens: 859
Total cost: 0.0044075


In [55]:
quiz = response.get('quiz')

In [57]:
quiz = quiz.replace("###RESPONSE_JSON", "").replace("\n", "")
quiz = json.loads(quiz)

In [51]:
##quiz = json.dumps(quiz)

In [58]:
quiz_table_data=[]
for key, value in quiz.items():
    mcq = value['mcq']
    options =" | ".join(
        [
            f"{option} : {option_value}" 
            for option,option_value in value['options'].items()
            ]    
        )
    correct = value['correct']
    quiz_table_data.append({'MCQ':mcq,"Choices":options,"Correct":correct})

In [60]:
quiz=pd.DataFrame(quiz_table_data)

In [61]:
quiz.to_csv("machinelearning.csv",index=False)

In [ ]:
from date